In [1]:
import sys
import copy
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from einops import rearrange
from Utilities import *

In [2]:
File_C, FID_C  = 'Data/carevue.csv',    [211, 646, 618, 51, 8368]                  # File Path of Data, Feature IDs [for carevue]
File_M, FID_M  = 'Data/metavision.csv', [220045, 220210, 220277, 220181, 220179]   # File Path of Data, Feature IDs [for metavision]
File_O = 'Model/4-EDSTE/'     # File path for outputs

W     = 3             # Window Size
UP    = 50            # Update Period
EPOC  = 50            # Number of epoch
alpha = 0.2           # Weight of heterogeneous transfer

NF_C  = len(FID_C)-1  # Number of features
NF_M  = len(FID_M)-1  # Number of features
loss_mae = torch.nn.L1Loss()
loss_mse = torch.nn.MSELoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
class AE(torch.nn.Module):
    def __init__(self, n_time):
        super(AE, self).__init__()
        ed0, ed1, ed2, ed3 = 16, 256, 64, 16
        self.Head = torch.nn.Sequential(torch.nn.Linear(n_time, ed0), torch.nn.Sigmoid(), torch.nn.Linear(ed0, ed1), torch.nn.Sigmoid(),torch.nn.Linear(ed1, ed2), torch.nn.LeakyReLU(), torch.nn.Linear(ed2, ed3), torch.nn.LeakyReLU(), torch.nn.Linear(ed3, 1))
    def forward(self, x):
        embv = self.Head(x)
        return embv

class PL(torch.nn.Module):
    def __init__(self, n_time, n_fea):
        super(PL, self).__init__()
        ed0, ed1, ed2, ed3 = 16, 256, 64, 16
        self.LinearS = torch.nn.Sequential(torch.nn.Linear(n_fea*n_time, ed0), torch.nn.Sigmoid(), torch.nn.Linear(ed0, ed1), torch.nn.Sigmoid(),torch.nn.Linear(ed1, ed2), torch.nn.LeakyReLU(), torch.nn.Linear(ed2, ed3), torch.nn.LeakyReLU(), torch.nn.Linear(ed3, n_time))
        self.Pred    = torch.nn.Sequential(torch.nn.Linear(n_fea+n_time, ed0), torch.nn.Sigmoid(), torch.nn.Linear(ed0, ed1), torch.nn.Sigmoid(),torch.nn.Linear(ed1, ed2), torch.nn.LeakyReLU(), torch.nn.Linear(ed2, ed3), torch.nn.LeakyReLU(), torch.nn.Linear(ed3,      1))
    def forward(self, xd, xs):
        S = self.LinearS(rearrange(xs, 'b f e -> b (f e)'))
        return self.Pred( torch.cat((xd, S), dim=1) )

In [4]:
NPara  = 0
Net_AE, Net_PL = AE(n_time=W), PL(n_time=W, n_fea=NF_C)
paras  = filter(lambda p: p.requires_grad, Net_AE.parameters())
NPara += sum([np.prod(p.size()) for p in paras])*NF_C
paras  = filter(lambda p: p.requires_grad, Net_PL.parameters())
NPara += sum([np.prod(p.size()) for p in paras])
print('Parameters:', NPara)

Parameters: 131768


### Training

In [ ]:
for Target in [1]:     # range(5)
    Target_C, Target_M = Target, Target
    XSC_Tra,XDC_Tra,YC_Tra,BC_Tra, XSC_Val,XDC_Val,YC_Val,BC_Val, XSC_Tes,XDC_Tes,YC_Tes,BC_Tes = load_data_SD(File_C, [0.6, 0.2, 0.2], FID_C, Target_C, W, device)     # Load Data
    XSM_Tra,XDM_Tra,YM_Tra,BM_Tra, XSM_Val,XDM_Val,YM_Val,BM_Val, XSM_Tes,XDM_Tes,YM_Tes,BM_Tes = load_data_SD(File_M, [0.6, 0.2, 0.2], FID_M, Target_M, W, device)     # Load Data
    Tnet_AEC, Tnet_PLC = [AE(n_time=W) for F in range(NF_C)], PL(n_time=W, n_fea=NF_C)     # Temporary Models (for Care)
    Tnet_AEM, Tnet_PLM = [AE(n_time=W) for F in range(NF_M)], PL(n_time=W, n_fea=NF_C)     # Temporary Models (for Meta)
    Snet_AEC, Snet_PLC = [AE(n_time=W) for F in range(NF_C)], PL(n_time=W, n_fea=NF_C)     # Saved Models (for Care)
    Snet_AEM, Snet_PLM = [AE(n_time=W) for F in range(NF_M)], PL(n_time=W, n_fea=NF_C)     # Saved Models (for Meta)
    for IterID in range(5):     # range(5)
        print("==========", Target, '-', IterID, "==========")
        # Initialization
        lue_C, lue_M, lul_C, lul_M = 0, 0, 0, 0     # last updated epoch & loss
        BL_C, PATH_C = np.inf, File_O+'Model-care-'+str(Target_C)+'-'+str(Target_M)+'_'+str(IterID)    # Best Loss, Model Saved Path
        BL_M, PATH_M = np.inf, File_O+'Model-meta-'+str(Target_C)+'-'+str(Target_M)+'_'+str(IterID)
        net_AEC, net_PLC = [AE(n_time=W) for F in range(NF_C)], PL(n_time=W, n_fea=NF_C)   # Model
        net_AEM, net_PLM = [AE(n_time=W) for F in range(NF_M)], PL(n_time=W, n_fea=NF_C)
        for F in range(NF_C): net_AEC[F].to(device)
        for F in range(NF_M): net_AEM[F].to(device)
        net_PLC.to(device)
        net_PLM.to(device)

        for epoc in range(EPOC):
            for d in range(2):     # 0=Train, 1=Valid
                if d==0:     # Train
                    XDC, XSC, YC, BC = XDC_Tra, XSC_Tra, YC_Tra, BC_Tra
                    XDM, XSM, YM, BM = XDM_Tra, XSM_Tra, YM_Tra, BM_Tra
                if d==1:     # Valid
                    XDC, XSC, YC, BC = XDC_Val, XSC_Val, YC_Val, BC_Val
                    XDM, XSM, YM, BM = XDM_Val, XSM_Val, YM_Val, BM_Val
                    Snet_AEC, Snet_PLC = copy.deepcopy(net_AEC), copy.deepcopy(net_PLC)     # Save the origin model
                    Snet_AEM, Snet_PLM = copy.deepcopy(net_AEM), copy.deepcopy(net_PLM)     # Save the origin model
                optimizer_HC, optimizer_RC = [torch.optim.Adam(net_AEC[F].parameters(), lr=0.0025) for F in range(NF_C)], torch.optim.Adam(net_PLC.parameters(), lr=0.01)
                optimizer_HM, optimizer_RM = [torch.optim.Adam(net_AEM[F].parameters(), lr=0.0025) for F in range(NF_M)], torch.optim.Adam(net_PLM.parameters(), lr=0.01)
                Pred_C, SL_C = torch.FloatTensor([]).to(device), []
                Pred_M, SL_M = torch.FloatTensor([]).to(device), []
                NRC, luf_C   = [BC[i+1]-BC[i] for i in range(len(BC)-1)], 0                 # Number of records of each patient, Flag for update
                NRM, luf_M   = [BM[i+1]-BM[i] for i in range(len(BM)-1)], 0

                for b in range( max([max(NRC), max(NRM)])//UP ):
                    if d==0 and b%5==4: print(epoc, 'Tra', b+1, '/', max([max(NRC), max(NRM)])//UP, end='     \r')
                    if d==1 and b%5==4: print(epoc, 'Val', b+1, '/', max([max(NRC), max(NRM)])//UP, end='     \r')
                    sl_C, sl_M = [], []
                    for p in range(len(BC)-1):
                        if NRC[p]>=UP*(b+1): sl_C += [BC[p]+UP*b+i for i in range(UP)]
                    for p in range(len(BM)-1):
                        if NRM[p]>=UP*(b+1): sl_M += [BM[p]+UP*b+i for i in range(UP)]        

                    # Heterogeneous Transfer (Training Phase Only)
                    if epoc-lue_C>3:
                        # Find the similar AE for Care from Meta
                        if b==0: print('Heterogeneous Transfer for Care at Epoc', epoc)
                        Tnet_AEC, Tnet_AEM, SAE_C, luf_C = copy.deepcopy(net_AEC), copy.deepcopy(net_AEM), [], 1
                        for FC in range(NF_C):
                            LL = []   # Loss list
                            for FM in range(NF_M):
                                ev = net_AEM[FM](XDC[sl_C,FC,:])
                                LL.append(float(loss_mae(torch.reshape(ev.detach(),(-1,)), YC[sl_C])))
                            SAE_C.append(np.argmin(LL))
                        for F in range(NF_C): 
                            sdC = Tnet_AEC[F].state_dict()
                            sdM = Tnet_AEM[SAE_C[F]].state_dict()
                            for key in sdC: sdC[key] = (1-alpha)*sdC[key] + alpha*sdM[key]
                            net_AEC[F].load_state_dict(sdC)
                        del ev
                        torch.cuda.empty_cache()
                    if epoc-lue_M>3:
                        # Find the similar AE for Meta from Care
                        if b==0: print('Heterogeneous Transfer for Meta at Epoc', epoc)
                        Tnet_AEC, Tnet_AEM, SAE_M, luf_M = copy.deepcopy(net_AEC), copy.deepcopy(net_AEM), [], 1
                        for FM in range(NF_M):
                            LL = []   # Loss list
                            for FC in range(NF_C):
                                ev = net_AEC[FC](XDM[sl_M,FM,:])
                                LL.append(float(loss_mae(torch.reshape(ev.detach(),(-1,)), YM[sl_M])))
                            SAE_M.append(np.argmin(LL))
                        for F in range(NF_M):
                            sdC = Tnet_AEC[SAE_M[F]].state_dict()
                            sdM = Tnet_AEM[F].state_dict()
                            for key in sdM: sdM[key] = (1-alpha)*sdM[key] + alpha*sdC[key]
                            net_AEM[F].load_state_dict(sdM)
                        del ev
                        torch.cuda.empty_cache()
                            
                    # Care            
                    EV = net_AEC[0]( XDC[sl_C,0,:] )
                    for FC in range(1,NF_C):
                        ev = net_AEC[FC]( XDC[sl_C,FC,:] )
                        EV = torch.cat((EV,ev), dim=1)
                    pred = net_PLC(EV, XSC[sl_C])
                    for FC in range(NF_C):       # Update AEs
                        loss = loss_mse(EV[:,FC], YC[sl_C])
                        loss.backward(retain_graph=True)
                    loss = loss_mse(torch.reshape(pred,(-1,)), YC[sl_C])
                    loss.backward()
                    for FC in range(NF_C): optimizer_HC[FC].step()
                    for FC in range(NF_C): optimizer_HC[FC].zero_grad()
                    optimizer_RC.step()
                    optimizer_RC.zero_grad()
                    Pred_C = torch.cat((Pred_C,torch.reshape(pred,(-1,)).detach()))
                    SL_C  += sl_C
                    del EV, ev, pred, loss
                    torch.cuda.empty_cache()
                    
                    # Meta
                    EV = net_AEM[0]( XDM[sl_M,0,:] )
                    for FM in range(1,NF_M):
                        ev = net_AEM[FM]( XDM[sl_M,FM,:] )
                        EV = torch.cat((EV,ev), dim=1)
                    pred = net_PLM(EV, XSM[sl_M])
                    for FM in range(NF_M):       # Update AEs
                        loss = loss_mae(EV[:,FM], YM[sl_M])
                        loss.backward(retain_graph=True)
                    loss = loss_mse(torch.reshape(pred,(-1,)), YM[sl_M])
                    loss.backward()
                    for FM in range(NF_M): optimizer_HM[FM].step()
                    for FM in range(NF_M): optimizer_HM[FM].zero_grad()
                    optimizer_RM.step()
                    optimizer_RM.zero_grad()
                    Pred_M = torch.cat((Pred_M,torch.reshape(pred,(-1,)).detach()))
                    SL_M  += sl_M
                    del EV, ev, pred, loss
                    torch.cuda.empty_cache()

                if d==1:     # Save Best (Validation Phase Only)
                    loss_C, loss_M = loss_mse(Pred_C, YC[SL_C]), loss_mse(Pred_M, YM[SL_M])
                    if epoc==0 or loss_C<lul_C or luf_C==1:   lul_C, lue_C = loss_C, epoc
                    if epoc==0 or loss_M<lul_M or luf_M==1:   lul_M, lue_M = loss_M, epoc
                    if loss_C<BL_C:
                        BL_C = loss_C
                        torch.save([Snet_PLC.state_dict()]+[Snet_AEC[F].state_dict() for F in range(NF_C)], PATH_C+'.pt') 
                        print('Save Best Care at', epoc, 'with loss of',  round(float(loss_C),3))
                    if loss_M<BL_M:
                        BL_M = loss_M
                        torch.save([Snet_PLM.state_dict()]+[Snet_AEM[F].state_dict() for F in range(NF_M)], PATH_M+'.pt') 
                        print('Save Best Meta at', epoc, 'with loss of',  round(float(loss_M),3))
                    if epoc%10==9: print(epoc+1, '/', EPOC, ' ', round(float(loss_C),3), round(float(loss_M),3), end='\r')

### Losses

In [6]:
for Target in range(5):     # range(5)
    Target_C,  Target_M  = Target, Target
    TVTLoss_C, TVTLoss_M = np.zeros((3,5)), np.zeros((3,5))     # Loss of train/valid/test
    XSC_Tra,XDC_Tra,YC_Tra,BC_Tra, XSC_Val,XDC_Val,YC_Val,BC_Val, XSC_Tes,XDC_Tes,YC_Tes,BC_Tes = load_data_SD(File_C, [0.6, 0.2, 0.2], FID_C, Target_C, W, device)     # Load Data
    XSM_Tra,XDM_Tra,YM_Tra,BM_Tra, XSM_Val,XDM_Val,YM_Val,BM_Val, XSM_Tes,XDM_Tes,YM_Tes,BM_Tes = load_data_SD(File_M, [0.6, 0.2, 0.2], FID_M, Target_M, W, device)
    Tnet_AEC, Tnet_PLC = [AE(n_time=W) for F in range(NF_C)], PL(n_time=W, n_fea=NF_C)     # Temporary Models (for Care)
    Tnet_AEM, Tnet_PLM = [AE(n_time=W) for F in range(NF_M)], PL(n_time=W, n_fea=NF_C)     # Temporary Models (for Meta)
    Snet_AEC, Snet_PLC = [AE(n_time=W) for F in range(NF_C)], PL(n_time=W, n_fea=NF_C)     # Saved Models (for Care)
    Snet_AEM, Snet_PLM = [AE(n_time=W) for F in range(NF_M)], PL(n_time=W, n_fea=NF_C)     # Saved Models (for Meta)
    net_AEM,   net_PLM = [AE(n_time=W) for F in range(NF_M)], PL(n_time=W, n_fea=NF_C)     # Model (for Care)
    net_AEC,   net_PLC = [AE(n_time=W) for F in range(NF_C)], PL(n_time=W, n_fea=NF_C)     # Model (for Meta)
    for IterID in range(5):     # range(5)
        print("==========", Target, '-', IterID, "==========")
        # Initialization
        PATH_C = File_O+'Model-care-'+str(Target_C)+'-'+str(Target_M)+'_'+str(IterID)     # Model Saved Path
        PATH_M = File_O+'Model-meta-'+str(Target_C)+'-'+str(Target_M)+'_'+str(IterID)
        # Load Saved Weightes
        Weights_C = torch.load(PATH_C+'.pt')
        Weights_M = torch.load(PATH_M+'.pt')  
        Snet_PLC.load_state_dict(Weights_C[0])
        Snet_PLM.load_state_dict(Weights_M[0])
        for F in range(NF_C): Snet_AEC[F].load_state_dict(Weights_C[1+F])
        for F in range(NF_M): Snet_AEM[F].load_state_dict(Weights_M[1+F])

        for d in range(1,3):     # 0=Train, 1=Valid, 2=Test
            if d==0: continue
            # Load Saved Weightes
            net_PLC, net_PLM = copy.deepcopy(Snet_PLC), copy.deepcopy(Snet_PLM)
            for F in range(NF_C): net_AEC[F] = copy.deepcopy(Snet_AEC[F])
            for F in range(NF_M): net_AEM[F] = copy.deepcopy(Snet_AEM[F])
            for F in range(NF_C): net_AEC[F].to(device)
            for F in range(NF_M): net_AEM[F].to(device)
            net_PLC.to(device)
            net_PLM.to(device)
            if d==0: XDC,XSC,YC,BC, XDM,XSM,YM,BM = XDC_Tra,XSC_Tra,YC_Tra,BC_Tra, XDM_Tra,XSM_Tra,YM_Tra,BM_Tra     # Training
            if d==1: XDC,XSC,YC,BC, XDM,XSM,YM,BM = XDC_Val,XSC_Val,YC_Val,BC_Val, XDM_Val,XSM_Val,YM_Val,BM_Val     # Validation
            if d==2: XDC,XSC,YC,BC, XDM,XSM,YM,BM = XDC_Tes,XSC_Tes,YC_Tes,BC_Tes, XDM_Tes,XSM_Tes,YM_Tes,BM_Tes     # Testing
            optimizer_HC, optimizer_RC = [torch.optim.Adam(net_AEC[F].parameters(), lr=0.0025) for F in range(NF_C)], torch.optim.Adam(net_PLC.parameters(), lr=0.01)
            optimizer_HM, optimizer_RM = [torch.optim.Adam(net_AEM[F].parameters(), lr=0.0025) for F in range(NF_M)], torch.optim.Adam(net_PLM.parameters(), lr=0.01)
            Pred_C, SL_C = torch.FloatTensor([]).to(device), []
            Pred_M, SL_M = torch.FloatTensor([]).to(device), []
            NRC, NRM     = [BC[i+1]-BC[i] for i in range(len(BC)-1)], [BM[i+1]-BM[i] for i in range(len(BM)-1)]      # Number of records of each patient, Flag for update

            for b in range( max([max(NRC), max(NRM)])//UP ):
                if d==0 and b%5==4: print('Tra', b+1, '/', max([max(NRC), max(NRM)])//UP, end='     \r')
                if d==1 and b%5==4: print('Val', b+1, '/', max([max(NRC), max(NRM)])//UP, end='     \r')
                if d==2 and b%5==4: print('Tes', b+1, '/', max([max(NRC), max(NRM)])//UP, end='     \r')
                sl_C, sl_M = [], []
                for p in range(len(BC)-1):
                    if NRC[p]>=UP*(b+1): sl_C += [BC[p]+UP*b+i for i in range(UP)]
                for p in range(len(BM)-1):
                    if NRM[p]>=UP*(b+1): sl_M += [BM[p]+UP*b+i for i in range(UP)]
                        
                # Heterogeneous Transfer (Training Phase Only)
                if True:
                    # Find the similar AE for Care from Meta
#                     if b==0: print('Heterogeneous Transfer for Care at Epoc', epoc)
                    Tnet_AEC, Tnet_AEM, SAE_C, luf_C = copy.deepcopy(net_AEC), copy.deepcopy(net_AEM), [], 1
                    for FC in range(NF_C):
                        LL = []   # Loss list
                        for FM in range(NF_M):
                            ev = net_AEM[FM](XDC[sl_C,FC,:])
                            LL.append(float(loss_mae(torch.reshape(ev.detach(),(-1,)), YC[sl_C])))
                        SAE_C.append(np.argmin(LL))
                    for F in range(NF_C): 
                        sdC = Tnet_AEC[F].state_dict()
                        sdM = Tnet_AEM[SAE_C[F]].state_dict()
                        for key in sdC: sdC[key] = (1-alpha)*sdC[key] + alpha*sdM[key]
                        net_AEC[F].load_state_dict(sdC)
                    del ev
                    torch.cuda.empty_cache()
                if True:
                    # Find the similar AE for Meta from Care
#                     if b==0: print('Heterogeneous Transfer for Meta at Epoc', epoc)
                    Tnet_AEC, Tnet_AEM, SAE_M, luf_M = copy.deepcopy(net_AEC), copy.deepcopy(net_AEM), [], 1
                    for FM in range(NF_M):
                        LL = []   # Loss list
                        for FC in range(NF_C):
                            ev = net_AEC[FC](XDM[sl_M,FM,:])
                            LL.append(float(loss_mae(torch.reshape(ev.detach(),(-1,)), YM[sl_M])))
                        SAE_M.append(np.argmin(LL))
                    for F in range(NF_M):
                        sdC = Tnet_AEC[SAE_M[F]].state_dict()
                        sdM = Tnet_AEM[F].state_dict()
                        for key in sdM: sdM[key] = (1-alpha)*sdM[key] + alpha*sdC[key]
                        net_AEM[F].load_state_dict(sdM)
                    del ev
                    torch.cuda.empty_cache()
                
                # Care            
                EV = net_AEC[0]( XDC[sl_C,0,:] )
                for FC in range(1,NF_C):
                    ev = net_AEC[FC]( XDC[sl_C,FC,:] )
                    EV = torch.cat((EV,ev), dim=1)
                pred = net_PLC(EV, XSC[sl_C])
                for FC in range(NF_C):       # Update AEs
                    loss = loss_mse(EV[:,FC], YC[sl_C])
                    loss.backward(retain_graph=True)
                loss = loss_mse(torch.reshape(pred,(-1,)), YC[sl_C])
                loss.backward()
                for FC in range(NF_C): optimizer_HC[FC].step()
                for FC in range(NF_C): optimizer_HC[FC].zero_grad()
                optimizer_RC.step()
                optimizer_RC.zero_grad()
                Pred_C = torch.cat((Pred_C,torch.reshape(pred,(-1,)).detach()))
                SL_C  += sl_C
                del EV, ev, pred, loss
                torch.cuda.empty_cache()
                    
                # Meta
                EV = net_AEM[0]( XDM[sl_M,0,:] )
                for FM in range(1,NF_M):
                    ev = net_AEM[FM]( XDM[sl_M,FM,:] )
                    EV = torch.cat((EV,ev), dim=1)
                pred = net_PLM(EV, XSM[sl_M])
                for FM in range(NF_M):       # Update AEs
                    loss = loss_mae(EV[:,FM], YM[sl_M])
                    loss.backward(retain_graph=True)
                loss = loss_mse(torch.reshape(pred,(-1,)), YM[sl_M])
                loss.backward()
                for FM in range(NF_M): optimizer_HM[FM].step()
                for FM in range(NF_M): optimizer_HM[FM].zero_grad()
                optimizer_RM.step()
                optimizer_RM.zero_grad()
                Pred_M = torch.cat((Pred_M,torch.reshape(pred,(-1,)).detach()))
                SL_M  += sl_M
                del EV, ev, pred, loss
                torch.cuda.empty_cache()

            loss_C, loss_M = loss_mse(Pred_C, YC[SL_C]), loss_mse(Pred_M, YM[SL_M])
            TVTLoss_C[d][IterID] = float(loss_C)
            TVTLoss_M[d][IterID] = float(loss_M)
#         print('Care')
        print(round(float(TVTLoss_C[0][IterID]),3))
        print(round(float(TVTLoss_C[1][IterID]),3))
        print(round(float(TVTLoss_C[2][IterID]),3))
#         print('Meta')
        print(round(float(TVTLoss_M[0][IterID]),3))
        print(round(float(TVTLoss_M[1][IterID]),3))
        print(round(float(TVTLoss_M[2][IterID]),3))
    PATH_C = File_O+'Loss-care-'+str(Target_C)+'-'+str(Target_M)
    PATH_M = File_O+'Loss-meta-'+str(Target_C)+'-'+str(Target_M)
    np.save(PATH_C, TVTLoss_C)
    np.save(PATH_M, TVTLoss_M)
    print('----------')
#     print('Care')
    print(round(np.mean(TVTLoss_C[0]),3))
    print(round(np.mean(TVTLoss_C[1]),3))
    print(round(np.mean(TVTLoss_C[2]),3))
#     print('Meta')
    print(round(np.mean(TVTLoss_M[0]),3))
    print(round(np.mean(TVTLoss_M[1]),3))
    print(round(np.mean(TVTLoss_M[2]),3))

Reading File ...


C:\Users\user\Desktop\Jia-Hao\KDD-Medical\Utilities.py:121: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  DF = pd.read_csv(File).values.tolist()


Finish Reading
torch.Size([1021159, 4, 3]) torch.Size([345050, 4, 3]) torch.Size([337918, 4, 3])
torch.Size([1021159, 4, 3]) torch.Size([345050, 4, 3]) torch.Size([337918, 4, 3])
torch.Size([1021159]) torch.Size([345050]) torch.Size([337918])
(2491,) (831,) (831,)
Reading File ...
Finish Reading
torch.Size([856154, 4, 3]) torch.Size([252348, 4, 3]) torch.Size([266293, 4, 3])
torch.Size([856154, 4, 3]) torch.Size([252348, 4, 3]) torch.Size([266293, 4, 3])
torch.Size([856154]) torch.Size([252348]) torch.Size([266293])
(1201,) (401,) (400,)
========== 0 - 0 ==========
0.0 875 / 878     
307.437
313.213
0.0
852.596
329.888
========== 0 - 1 ==========
0.0 875 / 878     
305.54
310.349
0.0
851.456
328.236
========== 0 - 2 ==========
0.0 875 / 878     
308.262
313.972
0.0
852.617
329.287
========== 0 - 3 ==========
0.0 875 / 878     
306.436
311.135
0.0
850.966
326.059
========== 0 - 4 ==========
0.0 875 / 878     
304.412
308.819
0.0
852.677
328.015
----------
0.0
306.417
311.498
0.0
852.062

C:\Users\user\Desktop\Jia-Hao\KDD-Medical\Utilities.py:121: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  DF = pd.read_csv(File).values.tolist()


Finish Reading
torch.Size([1031562, 4, 3]) torch.Size([350846, 4, 3]) torch.Size([342427, 4, 3])
torch.Size([1031562, 4, 3]) torch.Size([350846, 4, 3]) torch.Size([342427, 4, 3])
torch.Size([1031562]) torch.Size([350846]) torch.Size([342427])
(2491,) (831,) (831,)
Reading File ...
Finish Reading
torch.Size([848271, 4, 3]) torch.Size([248533, 4, 3]) torch.Size([264057, 4, 3])
torch.Size([848271, 4, 3]) torch.Size([248533, 4, 3]) torch.Size([264057, 4, 3])
torch.Size([848271]) torch.Size([248533]) torch.Size([264057])
(1201,) (401,) (400,)
========== 1 - 0 ==========
0.0 870 / 873     
17.239
18.386
0.0
45.755
40.319
========== 1 - 1 ==========
0.0 870 / 873     
17.867
18.994
0.0
46.103
40.479
========== 1 - 2 ==========
0.0 870 / 873     
16.608
17.767
0.0
47.385
41.545
========== 1 - 3 ==========
0.0 870 / 873     
17.969
18.7
0.0
47.632
41.679
========== 1 - 4 ==========
0.0 870 / 873     
19.776
20.562
0.0
45.677
40.327
----------
0.0
17.892
18.882
0.0
46.51
40.87
Reading File ...


...

Finish Reading
torch.Size([997128, 4, 3]) torch.Size([337003, 4, 3]) torch.Size([329190, 4, 3])
torch.Size([997128, 4, 3]) torch.Size([337003, 4, 3]) torch.Size([329190, 4, 3])
torch.Size([997128]) torch.Size([337003]) torch.Size([329190])
(2491,) (831,) (831,)
Reading File ...
Finish Reading
torch.Size([826437, 4, 3]) torch.Size([241157, 4, 3]) torch.Size([257297, 4, 3])
torch.Size([826437, 4, 3]) torch.Size([241157, 4, 3]) torch.Size([257297, 4, 3])
torch.Size([826437]) torch.Size([241157]) torch.Size([257297])
(1201,) (401,) (400,)
========== 2 - 0 ==========
0.0 870 / 871     
209.466
217.225
0.0
175020160.0
774.504
========== 2 - 1 ==========
0.0 870 / 871     
213.903
222.635
0.0
175020192.0
773.178
========== 2 - 2 ==========
0.0 870 / 871     
214.06
221.932
0.0
175020160.0
773.655
========== 2 - 3 ==========
0.0 870 / 871     
226.917
234.665
0.0
175020144.0
774.546
========== 2 - 4 ==========
0.0 870 / 871     
126.973
129.946
0.0
175020208.0
774.664
----------
0.0
198.264
20

C:\Users\user\Desktop\Jia-Hao\KDD-Medical\Utilities.py:121: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  DF = pd.read_csv(File).values.tolist()


Finish Reading
torch.Size([793192, 4, 3]) torch.Size([275046, 4, 3]) torch.Size([266026, 4, 3])
torch.Size([793192, 4, 3]) torch.Size([275046, 4, 3]) torch.Size([266026, 4, 3])
torch.Size([793192]) torch.Size([275046]) torch.Size([266026])
(2491,) (831,) (831,)
Reading File ...
Finish Reading
torch.Size([272576, 4, 3]) torch.Size([91423, 4, 3]) torch.Size([86707, 4, 3])
torch.Size([272576, 4, 3]) torch.Size([91423, 4, 3]) torch.Size([86707, 4, 3])
torch.Size([272576]) torch.Size([91423]) torch.Size([86707])
(1201,) (401,) (400,)
========== 3 - 0 ==========
0.0 70 / 71       
638.856
704.355
0.0
1370.204
1149.038
========== 3 - 1 ==========
0.0 70 / 71       
637.575
704.953
0.0
1372.535
1151.414
========== 3 - 2 ==========
0.0 70 / 71       
640.334
703.53
0.0
1366.665
1145.781
========== 3 - 3 ==========
0.0 70 / 71       
641.635
702.09
0.0
1366.068
1145.639
========== 3 - 4 ==========
0.0 70 / 71       
637.878
704.223
0.0
1366.716
1145.747
----------
0.0
639.256
703.83
0.0
1368.438

C:\Users\user\Desktop\Jia-Hao\KDD-Medical\Utilities.py:121: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  DF = pd.read_csv(File).values.tolist()


Finish Reading
torch.Size([795046, 4, 3]) torch.Size([275650, 4, 3]) torch.Size([266641, 4, 3])
torch.Size([795046, 4, 3]) torch.Size([275650, 4, 3]) torch.Size([266641, 4, 3])
torch.Size([795046]) torch.Size([275650]) torch.Size([266641])
(2491,) (831,) (831,)
Reading File ...
Finish Reading
torch.Size([271470, 4, 3]) torch.Size([91014, 4, 3]) torch.Size([86235, 4, 3])
torch.Size([271470, 4, 3]) torch.Size([91014, 4, 3]) torch.Size([86235, 4, 3])
torch.Size([271470]) torch.Size([91014]) torch.Size([86235])
(1201,) (401,) (400,)
========== 4 - 0 ==========
0.0 70 / 71       
147.296
159.966
0.0
562.14
561.866
========== 4 - 1 ==========
0.0 70 / 71       
147.761
161.46
0.0
561.251
561.907
========== 4 - 2 ==========
0.0 70 / 71       
193.772
207.841
0.0
560.302
562.96
========== 4 - 3 ==========
0.0 70 / 71       
156.828
168.368
0.0
559.262
560.609
========== 4 - 4 ==========
0.0 70 / 71       
155.371
164.719
0.0
563.361
562.517
----------
0.0
160.206
172.471
0.0
561.263
561.972


In [ ]:
# TVTLoss_C = np.load('Model/4-EDSTE/Saved/Loss-care-'+str(1)+'-'+str(1)+'.npy')
# TVTLoss_M = np.load('Model/4-EDSTE/Saved/Loss-meta-'+str(1)+'-'+str(1)+'.npy')
# for i in range(5):
#     print(i, 'C')
#     print('Training   Loss', round(TVTLoss_C[0][i],3))
#     print('Validation Loss', round(TVTLoss_C[1][i],3))
#     print('Testing    Loss', round(TVTLoss_C[2][i],3))
#     print(i, 'M')
#     print('Training   Loss', round(TVTLoss_M[0][i],3))
#     print('Validation Loss', round(TVTLoss_M[1][i],3))
#     print('Testing    Loss', round(TVTLoss_M[2][i],3))
# print('-----')
# print('Care')
# print('Training   Loss', round(np.mean(TVTLoss_C[0]),3))
# print('Validation Loss', round(np.mean(TVTLoss_C[1]),3))
# print('Testing    Loss', round(np.mean(TVTLoss_C[2]),3))
# print('Meta')
# print('Training   Loss', round(np.mean(TVTLoss_M[0]),3))
# print('Validation Loss', round(np.mean(TVTLoss_M[1]),3))
# print('Testing    Loss', round(np.mean(TVTLoss_M[2]),3))